### Датасет

- `purchases_train.csv` - история покупок в розничном магазине с 21 октября 2003 года по 12 марта 2004 года.
- `purchases_test.csv` - покупки за следующую неделю, с 13 по 19 марта 2004 года). В этой выборке для каждого пользователя исключены товары, которые он уже покупал за период обучающей выборки. Так сделано, чтобы измерить, насколько модель хорошо предсказывает будущие покупки.
- `customers.csv` - пол клиентов (`Male` / `Female`), есть пропуски

### Бейзлайн решение
- на истории покупок обучается модель матричного разложения `implicit.AlternateLeastSquares`
- рекомендации сравниваются с покупками за следующую неделю по метрике `map@10`


### Задание
Улучшите модель с помощью информации о поле клиентов `customers.csv`

In [1]:
# you need implicit & ml_metrics packages to run this notebook
# pip install implicit ml_metrics

from pathlib import Path

import numpy as np
from ml_metrics import mapk
import scipy
import pandas as pd
import implicit

from src.load import DataProvider
from src.utils import get_model, transform_to_user_sex_csr_matrix, transform_to_item_user_csr_matrix, get_recommendations, get_purchases_by_customer

In [2]:
data_provider = DataProvider(data_directory=Path('./data'))
purchases_train = data_provider.get_purchases_train()
purchases_test = data_provider.get_purchases_test()
display(
    purchases_train.head(),
    purchases_test.head(),
)

,customer_id,product_id,datetime
0,8698595,12530,2004-03-10 22:18:43.497459200
1,13271885,7541,2004-03-06 02:24:43.209763200
2,16852746,13134,2004-03-10 01:03:09.598614400
3,16852746,6572,2004-03-04 16:45:16.522566400
4,14619070,4659,2004-03-12 13:29:35.011481600


,customer_id,product_id,datetime
0,1021292,6197,2004-03-18 13:35:19.145152000
1,11379978,4659,2004-03-19 18:51:31.887936000
2,13271885,5659,2004-03-14 05:47:21.544166400
3,13271885,1015,2004-03-15 14:41:19.702089601
4,12315337,12072,2004-03-19 10:39:17.148105600


Обучаем модель implicit.AlternateLeastSquares

In [3]:
data_provider = DataProvider(data_directory=Path('./data'))
item_users = transform_to_item_user_csr_matrix(data_provider.get_purchases_train())

# baseline model
model = get_model()
np.random.seed(42)
model.fit(item_users=item_users)

test_user_ids, test_purchases = get_purchases_by_customer(data_provider.get_purchases_test())
recommendations = get_recommendations(model, test_user_ids, item_users)
baseline_score = mapk(test_purchases, recommendations, k=10)
baseline_score


100%|██████████| 7.0/7 [00:04<00:00,  1.78it/s]


0.18293975547686622

In [12]:
#Let's load customers
customers = data_provider.get_customers()

#display the a few rows
display((
    customers.head()
))

#show basic info
print(customers.info())
#let's full nan values using "unknown"
customers.fillna("unknown", inplace=True)

,customer_id,sex
0,14386819,Female
1,1481405,NaN
2,16745074,NaN
3,10325906,NaN
4,11167384,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123565 entries, 0 to 123564
Data columns (total 2 columns):
customer_id    123565 non-null int64
sex            34476 non-null object
dtypes: int64(1), object(1)
memory usage: 1.9+ MB
None


In [13]:
# Let's creata a sparse matrix
# transform_to_user_sex_csr_matrix is created in utils.py
customer_options = transform_to_user_sex_csr_matrix(customers)

In [7]:
import scipy.sparse as sp
# we need to join data from item_users, customer_options
# to do that for sparse matrix we can use sp.vstack 
data = sp.vstack((item_users, customer_options))

model = get_model()
np.random.seed(42)
model.fit(item_users=data)

test_user_ids, test_purchases = get_purchases_by_customer(data_provider.get_purchases_test())
recommendations = get_recommendations(model, test_user_ids, data)
baseline_score = mapk(test_purchases, recommendations, k=10)
baseline_score

100%|██████████| 7.0/7 [00:05<00:00,  1.39it/s]


0.1967428658126718

The final score is better than the baseline result